In [1]:
import os
import base64
import requests
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [2]:
COGNITO_DOMAIN = "https://my-example-286574326306-domain.auth.us-east-1.amazoncognito.com/oauth2/token"
COGNITO_CLIENT_ID = 'g0vlne37nsmebenkoqjemfbr1'
COGNITO_CLIENT_SECRET = os.environ['COGNITO_CLIENT_SECRET']

# Terraform resource_server_id is usually formatted like: <user_pool_id>|<identifier>
RESOURCE_SERVER_ID = "my-api"
RESOURCE_SERVER_IDENTIFIER = RESOURCE_SERVER_ID.split("|", 1)[1] if "|" in RESOURCE_SERVER_ID else RESOURCE_SERVER_ID

SCOPES = [
    f"{RESOURCE_SERVER_ID}/read",
    f"{RESOURCE_SERVER_ID}/write",
]
SCOPE = ' '.join(SCOPES)

In [3]:
basic = base64.b64encode(
    f"{COGNITO_CLIENT_ID}:{COGNITO_CLIENT_SECRET}".encode("utf-8")
).decode("utf-8")

headers = {
    "Authorization": f"Basic {basic}",
    "Content-Type": "application/x-www-form-urlencoded",
}

data = {
    "grant_type": "client_credentials",
    "scope": SCOPE,
}

In [4]:
resp = requests.post(COGNITO_DOMAIN, headers=headers, data=data, timeout=30)

if resp.status_code != 200:
    print("❌ Failed to obtain token")
    print(resp.status_code, resp.text)
    raise SystemExit(1)

token_json = resp.json()
print("✅ Token acquired")
print("Token response keys:", list(token_json.keys()))
print("scope requested:", SCOPE)
print("access_token (first 40 chars):", token_json["access_token"][:40], "...")

✅ Token acquired
Token response keys: ['access_token', 'expires_in', 'token_type']
scope requested: my-api/read my-api/write
access_token (first 40 chars): eyJraWQiOiJSSER3WHM2Tm8xR0lBXC9kcnBvVnZE ...
